### VectorStoreRetrieverMemory

`VectorStoreRetrieverMemory`는 벡터 스토어에 메모리를 저장하고, 호출될 떄마다 가장 눈에 띄는 상위 K개의 문서를 쿼리한다.

이는 대화내용의 순서를 명시적으로 추적하지않는다는 점에서 다른 대부분의 메모리 클래스와 다르다. 

- `벡터스토어에 저장`하고, 상위 K개의 문서를 `query`한다. 시간순서를 고려하지않고, 검색을 한다.
- 대화가 오래 되었고, 오래 축적했을 때, 정보가 많이 석였을때, 검색을 통해 해결해야 하는데, 이때 유용하다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import faiss
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vector Store를 초기화한다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vector_store = FAISS(embeddings, index, InMemoryDocstore({}), {})

실제 사용에서는 k를 높은값을 설정하지만, 여기서는 k=1을 사용하여 다음과같이 표시. 

In [17]:
from langchain.memory import VectorStoreRetrieverMemory
retriever = vector_store.as_retriever(search_kwargs={"k":1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [18]:
# 임의의 대화를 저장합니다.
memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)
memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

In [19]:
# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(memory.load_memory_variables({"prompt": "면접자 전공은 무엇인가요?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.


In [20]:
print(
    memory.load_memory_variables(
        {"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"}
    )["history"]
)

human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.
